# Pythonic SysML Elements

This notebook walks through the approach to making SysML elements act Pythonically while in the PyMBE environment in order to support more intuitive codes that inspect and analyze SysML models.

Note that the Markdown headers for code snippets are set up to be translated into docstrings for documentation of the code eventually.

In [1]:
import json
from dataclasses import dataclass, field
from typing import Any, Dict, List, Set, Tuple, Union
from collections import defaultdict

from uuid import uuid4

with open('example_data/sysml_3a.json', 'r') as fp:
    element_raw_data = json.load(fp)
    
printing_level = "TRACE"

## Interchange data and loading

The API for standing up a model in memory is built around loading data from Python dictionaries. This is mostly based around major APIs of interest (e.g., the SysML v2 REST API) exchanging data in the JSON format with a main set of keys and values.

### Bulk loading model data

The entry point to loading up the models is a bulk load method that gathers collections of dictionaries (each model element is expected to serialize as a dictionary from JSON) and creates a new dictionary that allows for rapid look up of elements by their IDs.

In [2]:
def index_element_data(elements):
    return {element["@id"]: element for element in elements}

### ID reference format

The serialized JSON files will present IDs as a single entry dictionary of the form {'@id': uuid}.

In [3]:
index_element_data(element_raw_data)[element_raw_data[0]['@id']]

{'@type': 'Redefinition',
 '@id': '023b4d09-6bc3-4d70-b6d8-e6c9b4679d70',
 'aliasIds': [],
 'documentation': [],
 'effectiveName': None,
 'elementId': '023b4d09-6bc3-4d70-b6d8-e6c9b4679d70',
 'general': {'@id': '20205904-dcb9-487c-9b87-dd5e55e39396'},
 'isImplied': False,
 'isImpliedIncluded': False,
 'isLibraryElement': False,
 'name': None,
 'ownedAnnotation': [],
 'ownedElement': [],
 'ownedRelatedElement': [],
 'ownedRelationship': [],
 'owner': None,
 'owningFeature': {'@id': '56851b8c-56c7-4429-8bd0-17fc09b3ce02'},
 'owningMembership': None,
 'owningNamespace': None,
 'owningRelatedElement': {'@id': '56851b8c-56c7-4429-8bd0-17fc09b3ce02'},
 'owningRelationship': None,
 'owningType': {'@id': '56851b8c-56c7-4429-8bd0-17fc09b3ce02'},
 'qualifiedName': None,
 'redefinedFeature': {'@id': '20205904-dcb9-487c-9b87-dd5e55e39396'},
 'redefiningFeature': {'@id': '56851b8c-56c7-4429-8bd0-17fc09b3ce02'},
 'relatedElement': [{'@id': '56851b8c-56c7-4429-8bd0-17fc09b3ce02'},
  {'@id': '20205904

## Pythonic Model elements

The methods and the class developed here are intended to support the Python representation and manipulation of system model data. The partial methods describe below set up various fields in objects of the Element class for use in downstream or user codes involving this library.

There are a variety of features developed here to make the interaction with system model data more Pythonic.

### List of Items Accessible by Name

For convenience, a class is introduced that allows for collections of objects to be referenced by name rather than by unique identifier. This requires that names in a given list are unique.

#### Precursors

Classes that this class references are stubbed below for its use.

In [4]:
class Element():
    """Dummy version for algorithm testing"""
    _data = None
    _model = None

In [5]:
class Model():
    """Dummy version for algorithm testing"""
    elements: Dict[str, "Element"]

The ListOfNamedItemsClass extends list to make names within model element data a useable index.

In [6]:
class ListOfNamedItems(list):
    """A list that also can return items by their name."""
    
    # FIXME: Should really build/update the item map on entry modification rather than lookup
    
    # FIXME: figure out why __dir__ of returned objects think they are lists
    def __getitem__(self, key):
        item_map = {
            item._data["name"]: item
            for item in self
            if isinstance(item, Element) and "name" in item._data
        }
        effective_item_map = {
            item._data["effectiveName"]: item
            for item in self
            if isinstance(item, Element) and "effectiveName" in item._data
        }
        if key in item_map:
            return item_map[key]
        if key in effective_item_map:
            return effective_item_map[key]
        if isinstance(key, int):
            return super().__getitem__(key)
        return None

In [7]:
def is_id_item(item):
    return isinstance(item, dict) and item['@id'] is not None and isinstance(item['@id'], str)

In [8]:
is_id_item([])

False

In [9]:
is_id_item({'@id': '56851b8c-56c7-4429-8bd0-17fc09b3ce02'})

True

In [10]:
dummy_ele = Element()
is_id_item(dummy_ele)

False

### Navigating fields for owned elements (and more specialized collections)

Elements should be able to directly reference elements through ownership relations. This is done by dynamically constructing Python fields for meta-attributes that start with "owned."

In [11]:
def set_names_for_owned_for_element(self):
    data = self._data
    
    # iterate through the meta-attributes for the model element in serial form
    for key, items in data.items():
        if printing_level == "TRACE":
            print(f'Key is {key} and items are {items}')
        if key.startswith("owned") and isinstance(items, list):
            new_list = [self._model.elements[item['@id']] for item in items if is_id_item(item)]
            if printing_level == "TRACE" and len(new_list) > 0:
                print(f'Expanding {key} with new list of named items')
            if len(new_list) > 0:
                data[key] = ListOfNamedItems(new_list)

#### Tester

Build an object with only the necessary fields for the test from raw JSON data. In this case, this is an object with "_data" and "_model" fields, and then another object type with an "elements" field.

In [12]:
def construct_test_element_from_raw(raw_dict, model_object):
    test_element = Element()
    test_element._data = raw_dict
    test_element._model = model_object
    return test_element

In [13]:
test_model = Model()
indexed_raw = index_element_data(element_raw_data)
test_elements = {indx: construct_test_element_from_raw(raw_item, test_model)
                for indx, raw_item in indexed_raw.items()}
setattr(test_model, "elements", test_elements)

In [14]:
test_model.elements[element_raw_data[2]['@id']]._data

{'@type': 'ItemFlowEnd',
 '@id': '03deed71-fa6f-438f-abe5-aa0aed87ef71',
 'aliasIds': [],
 'chainingFeature': [],
 'differencingType': [],
 'directedFeature': [{'@id': 'a00075f8-bb12-4b6a-ba6a-07c9f6cec315'}],
 'direction': None,
 'documentation': [],
 'effectiveName': 'target',
 'elementId': '03deed71-fa6f-438f-abe5-aa0aed87ef71',
 'endFeature': [],
 'endOwningType': {'@id': 'ed250300-0077-42f6-b893-1cda693b72c2'},
 'feature': [{'@id': '56851b8c-56c7-4429-8bd0-17fc09b3ce02'},
  {'@id': 'a00075f8-bb12-4b6a-ba6a-07c9f6cec315'},
  {'@id': 'd997ec27-b1ed-4aca-bab5-488e4e80e5f0'}],
 'featureMembership': [{'@id': 'c4914f92-09f3-4a05-81de-b169eb1d22d7'},
  {'@id': '993b7a8d-2301-4d52-9b3c-194a773bdd1c'}],
 'featuringType': [{'@id': 'ed250300-0077-42f6-b893-1cda693b72c2'}],
 'importedMembership': [],
 'inheritedFeature': [{'@id': 'a00075f8-bb12-4b6a-ba6a-07c9f6cec315'},
  {'@id': 'd997ec27-b1ed-4aca-bab5-488e4e80e5f0'}],
 'inheritedMembership': [{'@id': '993b7a8d-2301-4d52-9b3c-194a773bdd1c'}

In [15]:
set_names_for_owned_for_element(test_model.elements[element_raw_data[2]['@id']])

Key is @type and items are ItemFlowEnd
Key is @id and items are 03deed71-fa6f-438f-abe5-aa0aed87ef71
Key is aliasIds and items are []
Key is chainingFeature and items are []
Key is differencingType and items are []
Key is directedFeature and items are [{'@id': 'a00075f8-bb12-4b6a-ba6a-07c9f6cec315'}]
Key is direction and items are None
Key is documentation and items are []
Key is effectiveName and items are target
Key is elementId and items are 03deed71-fa6f-438f-abe5-aa0aed87ef71
Key is endFeature and items are []
Key is endOwningType and items are {'@id': 'ed250300-0077-42f6-b893-1cda693b72c2'}
Key is feature and items are [{'@id': '56851b8c-56c7-4429-8bd0-17fc09b3ce02'}, {'@id': 'a00075f8-bb12-4b6a-ba6a-07c9f6cec315'}, {'@id': 'd997ec27-b1ed-4aca-bab5-488e4e80e5f0'}]
Key is featureMembership and items are [{'@id': 'c4914f92-09f3-4a05-81de-b169eb1d22d7'}, {'@id': '993b7a8d-2301-4d52-9b3c-194a773bdd1c'}]
Key is featuringType and items are [{'@id': 'ed250300-0077-42f6-b893-1cda693b72c2

In [16]:
test_model.elements[element_raw_data[2]['@id']]._data['ownedElement'][0]._data

{'@type': 'ReferenceUsage',
 '@id': '56851b8c-56c7-4429-8bd0-17fc09b3ce02',
 'aliasIds': [],
 'chainingFeature': [],
 'definition': [{'@id': '83893534-5d4f-48f4-ae5a-d6da8356ae53'}],
 'differencingType': [],
 'directedFeature': [],
 'directedUsage': [],
 'direction': None,
 'documentation': [],
 'effectiveName': 'engineTorque',
 'elementId': '56851b8c-56c7-4429-8bd0-17fc09b3ce02',
 'endFeature': [],
 'endOwningType': None,
 'feature': [],
 'featureMembership': [],
 'featuringType': [{'@id': '03deed71-fa6f-438f-abe5-aa0aed87ef71'}],
 'importedMembership': [],
 'inheritedFeature': [],
 'inheritedMembership': [],
 'input': [],
 'intersectingType': [],
 'isAbstract': False,
 'isComposite': False,
 'isConjugated': False,
 'isDerived': False,
 'isEnd': False,
 'isImpliedIncluded': False,
 'isLibraryElement': False,
 'isOrdered': False,
 'isPortion': False,
 'isReadOnly': False,
 'isReference': True,
 'isSufficient': False,
 'isUnique': True,
 'isVariation': False,
 'member': [],
 'membership

In [17]:
del Element, Model

### Accessing fields Pythonically

The next thing to add to the Element class are magics to match the meta-attributes of loaded elements to properties on Python objects. That is done by adding to the getattr and getitem magics.

This should focus on the metafields with expected literal values in the JSON.

#### More developed Model class

This iteration of the Model class adds the needed methods to support Pythonic relationships.

In [22]:
class Model:  # pylint: disable=too-many-instance-attributes
    """A SysML v2 Model"""

    # TODO: Look into making elements immutable (e.g., frozen dict)
    elements: Dict[str, "Element"]

    name: str = "SysML v2 Model"

    all_relationships: Dict[str, "Element"] = field(default_factory=dict)
    all_non_relationships: Dict[str, "Element"] = field(default_factory=dict)

    ownedElement: ListOfNamedItems = field(  # pylint: disable=invalid-name
        default_factory=ListOfNamedItems,
    )
    ownedMetatype: Dict[str, List["Element"]] = field(  # pylint: disable=invalid-name
        default_factory=dict,
    )
    ownedRelationship: List["Element"] = field(  # pylint: disable=invalid-name
        default_factory=list,
    )

    max_multiplicity = 100

    source: Any = None

#### More developed Element class

This iteration of the Element class adds the needed methods to support the Pythonic field accessors.

In [23]:
class Element():
    _data: dict
    _model: Model

    _id: str = field(default_factory=lambda: str(uuid4()))
    _metatype: str = "Element"
    _derived: Dict[str, List] = field(default_factory=lambda: defaultdict(list))
    
    def __getattr__(self, key: str):
        try:
            return self[key]
        except KeyError as exc:
            raise AttributeError(f"Cannot find {key}") from exc
    
    # FIXME: Take advantage of new JSON schema put out in the SysML specification data to isolate fields of most interest.
    
    def __getitem__(self, key: str):
        found = False
        for source in ("_data", "_derived"):
            source = self.__getattribute__(source)
            if key in source:
                if printing_level == 'TRACE':
                    print(f'Found meta-attribute {key}')
                found = True
                item = source[key]
                break
        if not found:
            raise KeyError(f"No '{key}' in {self}")

        if isinstance(item, (dict, str)):
            item = self.__safe_dereference(item)
        elif isinstance(item, (list, tuple, set)):
            items = [self.__safe_dereference(subitem) for subitem in item]
            return type(item)(items)
        return item
    
    def __safe_dereference(self, item):
        """If given a reference to another element, try to get that element"""
        try:
            if isinstance(item, dict) and "@id" in item:
                if len(item) > 1:
                    warn(f"Found a reference with more than one entry: {item}")
                item = item["@id"]
            return self._model.elements[item]
        except KeyError:
            return item

Demonstrate the application of the owned element resolution to all elements in the example data.

In [24]:
def construct_test_element_from_raw(raw_dict, model_object):
    test_element = Element()
    test_element._data = raw_dict
    test_element._model = model_object
    return test_element

In [25]:
with open('example_data/sysml_3a.json', 'r') as fp:
    element_raw_data2 = json.load(fp)

test_model2 = Model()
indexed_raw2 = index_element_data(element_raw_data2)
test_elements2 = {indx: construct_test_element_from_raw(raw_item, test_model2)
                for indx, raw_item in indexed_raw2.items()}
test_model2.elements = test_elements2

In [27]:
set_names_for_owned_for_element(test_model2.elements[element_raw_data[2]['@id']])

In [33]:
printing_level = "ERROR"

In [34]:
for element_raw in element_raw_data:
    set_names_for_owned_for_element(test_model2.elements[element_raw['@id']])

In [35]:
test_model2.elements[element_raw_data[2]['@id']]._metatype

'Element'

In [36]:
test_model2.elements[element_raw_data[2]['@id']].effectiveName

'target'

In [37]:
test_model2.elements[element_raw_data[2]['@id']].ownedElement['engineTorque']

In [38]:
test_model2.elements[element_raw_data[2]['@id']]

In [40]:
[{element_raw['name']: [ele['name'] for ele in test_model2.elements[element_raw['@id']].ownedElement]} for element_raw in element_raw_data]

[{None: []},
 {None: []},
 {None: [None]},
 {None: []},
 {None: []},
 {None: []},
 {None: ['3a-Function-based Behavior-1']},
 {None: []},
 {None: []},
 {None: [None]},
 {None: []},
 {'Definitions': ['FuelCmd',
   None,
   'EngineStart',
   'EngineOff',
   None,
   'Generate Torque',
   'Amplify Torque',
   'Transfer Torque',
   'Distribute Torque',
   'Provide Power']},
 {None: []},
 {None: []},
 {None: []},
 {None: []},
 {None: []},
 {None: []},
 {None: []},
 {None: []},
 {None: []},
 {None: []},
 {None: []},
 {None: []},
 {None: []},
 {None: []},
 {None: []},
 {None: []},
 {None: []},
 {None: []},
 {None: []},
 {None: []},
 {None: [None, None]},
 {None: []},
 {None: []},
 {None: []},
 {None: [None, None]},
 {None: []},
 {None: []},
 {None: []},
 {'Generate Torque': ['fuelCmd', 'engineTorque']},
 {'engineTorque': []},
 {None: []},
 {None: []},
 {None: []},
 {'engineStart': []},
 {None: []},
 {None: [None, None]},
 {None: []},
 {None: []},
 {None: []},
 {None: [None, None]},
 {None: []

### Navigating relationships Pythonically

The next Pythonic task for our model elements is the ability to traverse SysML v2 relationships Pythonically, either in the forward (through) or reverse direction with simple property accessors.

In [32]:
def element_resolve_to_determine_element_as_relationship(self):
    self._is_relationship = "relatedElement" in data

In [ ]:
 def model_add_relationships(self):
        """Adds relationships to elements"""
        relationship_mapper = {
            "through": ("source", "target"),
            "reverse": ("target", "source"),
        }
        # TODO: make this more elegant...  maybe.
        for relationship in self.all_relationships.values():
            endpoints = {
                endpoint_type: [
                    self.elements[endpoint["@id"]]
                    for endpoint in relationship._data[endpoint_type]
                ]
                for endpoint_type in ("source", "target")
            }
            metatype = relationship._metatype
            for direction, (key1, key2) in relationship_mapper.items():
                endpts1, endpts2 = endpoints[key1], endpoints[key2]
                for endpt1 in endpts1:
                    for endpt2 in endpts2:
                        endpt1._derived[f"{direction}{metatype}"] += [{"@id": endpt2._data["@id"]}]

### Dealing with derived attributes

In SysML v2 models, there are a large number of derived attributes and collections that can gather the incoming and outgoing relationships to other elements. These are actually the standard means of navigation rather than the relationship-based fields generated specifically for pyMBE (also to make it easier to remember some of the names).

### Creating model elements Pythonically

After setting up a model in memory Pythonically, the next steps will be to create model elements within the model processing session.

## Example: Working with Actions in SysML v2 test case 3a

### Fields for model element attributes

The fields for a given element should be referenceable in a Pythonic way, using the dot operator. In addition, the field name should indicate whether the reference is "inside" the Element or avaiable through a traversal reference. In this case, we use "through" for moving from source to target on a relationship and "reverse" to go from target to source.

In [2]:
all_action_defs = {ele_id:ele for (ele_id, ele)
                   in test_model.all_non_relationships.items()
                   if ele['@type'] == 'ActionDefinition'} 
all_action_defs

{'1fc1cef9-5ba6-4150-ab8a-ae483583eb18': <Generate Torque «ActionDefinition»>,
 '2f97f7f3-5085-4fb2-a8d4-9729db79e0af': <Transfer Torque «ActionDefinition»>,
 '4ce66471-d562-4b33-8d2b-c8232ad5884a': <Provide Power «ActionDefinition»>,
 '772cc577-68ef-4180-93a6-3e0b4c5459f2': <Distribute Torque «ActionDefinition»>,
 '8fe4fd3c-5618-46a2-8358-316560b49231': <Amplify Torque «ActionDefinition»>}

In [3]:
transfer_action = [ele for (ele_id, ele)
                   in all_action_defs.items() if ele['name'] == "Transfer Torque"][0]

With the object assigned to the variable above, we can now explore the element fields:

In [4]:
transfer_action.name

'Transfer Torque'

In [5]:
transfer_action.ownedElement

[<transmissionTorque «ReferenceUsage»>, <driveshaftTorque «ReferenceUsage»>]

In [6]:
transfer_action.ownedRelationship

[<FeatureMembership([<Transfer Torque «ActionDefinition»>] ←→ [<transmissionTorque «ReferenceUsage»>])>,
 <FeatureMembership([<Transfer Torque «ActionDefinition»>] ←→ [<driveshaftTorque «ReferenceUsage»>])>]

In [7]:
transfer_action.throughFeatureMembership[0]

<driveshaftTorque «ReferenceUsage»>

### Working with Derived Attributes

SysML v2 has a number of attributes that are derived from the Element's relationship to other Elements. For performance reasons, it may be desirable to evaluate these derivations lazily. It may also be desirable to precache these evaluations or build them up in the background. 

## Example: Working with Clean Bicycle KerML example

The same apporach to model load can work with KerML exports also with some refactoring effort.

In [8]:
with open('example_data/Clean_Bicycle.json', 'r') as fp:
    element_raw_data = json.load(fp)

In [9]:
len(element_raw_data)

578

In [10]:
extracted_elements = [(lambda d: d.update({'@id': ele['identity']['@id']}) or d)(ele['payload'])
                      for ele in element_raw_data]

In [11]:
bicycle_model = pm.Model.load(extracted_elements)

In [14]:
bicycle_model

<SysML v2 Model (578 elements)>

In [19]:
all_class_defs = {ele_id:ele for (ele_id, ele)
                   in bicycle_model.all_non_relationships.items()
                   if ele['@type'] == 'Classifier'} 
all_class_defs

{'87af228c-582f-47e7-b839-fd087651f365': <Anything «Classifier»>,
 'ade99188-b733-456e-b36e-da48eea6634e': <A Bike «Classifier»>,
 '5c76919e-54b8-4478-9e6e-5b185ebbf686': <A Seat «Classifier»>,
 '6d191eb7-6ac3-4367-ac9d-19a03c55be10': <Wheel 1 «Classifier»>,
 '70aca545-7c74-4f51-b86f-295d2fbbcc1e': <Wheel 2 «Classifier»>,
 'a707dc82-6838-4683-8f01-e85c5969e7e9': <A Bike «Classifier»>,
 'd483e836-71a7-4fa4-b12b-87ccbee88131': <A Seat «Classifier»>,
 '9347a3a9-1292-4572-a5e9-6f668977d6cf': <Wheel «Classifier»>,
 'bff8a8bd-5af0-408f-a1c6-fb5963b8dea3': <A Bike «Classifier»>,
 '38993def-3ac2-4d68-ab79-4292006baf4f': <A Seat «Classifier»>,
 'eec45ce3-7c16-47bd-8684-cbdb4537e50a': <Wheel 2 «Classifier»>,
 'b6e1d8bf-0866-4f04-a22b-5b2f3cd003ec': <Wheel 1 «Classifier»>,
 '776a0aba-638b-4d81-8ba6-0eb3318f3cbc': <Wheel «Classifier»>}

In [20]:
all_feature_defs = {ele_id:ele for (ele_id, ele)
                   in bicycle_model.all_non_relationships.items()
                   if ele['@type'] == 'Feature'}
all_feature_defs

{'3b94de7a-06c8-4788-a63b-44de4a238e10': <things «Feature»>,
 '66617c14-8e6f-451f-b48f-f5c30f2a30b9': <66617c14-8e6f-451f-b48f-f5c30f2a30b9 «Feature»>,
 '753bcd40-49e2-4caa-9e42-c0fdaae036db': <753bcd40-49e2-4caa-9e42-c0fdaae036db «Feature»>,
 '59eafeb0-480e-4913-ae24-6538b541324e': <naturals «Feature»>,
 '31de0e8f-1c60-4627-87ad-8a6a139a6d9e': <31de0e8f-1c60-4627-87ad-8a6a139a6d9e «Feature»>,
 '998e3fc9-3a42-4e5b-9057-2870479631a9': <998e3fc9-3a42-4e5b-9057-2870479631a9 «Feature»>,
 '5f2595d2-0c07-4ddf-aa7b-013d6b3d7498': <5f2595d2-0c07-4ddf-aa7b-013d6b3d7498 «Feature»>,
 'c68f4b5f-db8c-42dd-b711-459b16df19ce': <My Seat «Feature»>,
 '3396e57f-23cd-4fbe-a55d-8c4966d79897': <3396e57f-23cd-4fbe-a55d-8c4966d79897 «Feature»>,
 'e81c2f88-04b5-4c2a-aa46-0d1b671ee0df': <e81c2f88-04b5-4c2a-aa46-0d1b671ee0df «Feature»>,
 'e5126bfe-4b08-49d3-ab64-e0022e74fc4a': <e5126bfe-4b08-49d3-ab64-e0022e74fc4a «Feature»>,
 '948dd903-0cd0-4394-8fa6-d1e3ebec01d7': <948dd903-0cd0-4394-8fa6-d1e3ebec01d7 «Featur

In [22]:
all_ss_defs = [ele for (ele_id, ele)
                   in bicycle_model.all_relationships.items()
                   if ele['@type'] == 'Subsetting']
all_ss_defs

[<Subsetting([<753bcd40-49e2-4caa-9e42-c0fdaae036db «Feature»>] ←→ [<things «Feature»>])>,
 <Subsetting([<7f293023-91e2-473e-bd86-f33207ddb1af «LiteralInteger»>] ←→ [<literalIntegerEvaluations «Expression»>])>,
 <Subsetting([<11d21539-a652-41f1-a8e0-797892d718d0 «MultiplicityRange»>] ←→ [<naturals «Feature»>])>,
 <Subsetting([<31de0e8f-1c60-4627-87ad-8a6a139a6d9e «Feature»>] ←→ [<things «Feature»>])>,
 <Subsetting([<f6aec2a0-2f16-46d0-a879-fd9d87d6fc20 «LiteralInteger»>] ←→ [<literalIntegerEvaluations «Expression»>])>,
 <Subsetting([<3594ef6e-8ee0-4b9a-a4b8-14be786394a9 «MultiplicityRange»>] ←→ [<naturals «Feature»>])>,
 <Subsetting([<998e3fc9-3a42-4e5b-9057-2870479631a9 «Feature»>] ←→ [<things «Feature»>])>,
 <Subsetting([<2e273be6-d551-4ac5-8d8b-111666ed30d1 «LiteralInteger»>] ←→ [<literalIntegerEvaluations «Expression»>])>,
 <Subsetting([<39d0ae2c-e041-4161-b6f2-c6edbdc7ae3a «MultiplicityRange»>] ←→ [<naturals «Feature»>])>,
 <Subsetting([<5f2595d2-0c07-4ddf-aa7b-013d6b3d7498 «Featu